<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/JupyterNotebooks/solar_icf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import subprocess
running_on_colab = not "JupyterNotebooks" in subprocess.os.getcwd()
if running_on_colab:
    running_on_colab = True
    parent_dir = "/content/"
    export_data_dir = "/content/"
    import_data_dir = "/content/"
else:
    parent_dir = subprocess.os.getcwd().split("JupyterNotebooks")[0]
    sep_char = parent_dir.split("Energy")[-1]
    import_data_dir =  parent_dir + "ImportedData" + sep_char 
    export_data_dir =  parent_dir + "ExportedData" + sep_char
parent_dir, import_data_dir, export_data_dir

('c:\\Users\\engin\\Documents\\GitHub\\Energy\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\')